In [ ]:
import sqlalchemy
import sqlalchemy.ext.declarative
DB_NAME="sqlalchemy_levelup"

In [ ]:
! mysql -e "DROP DATABASE IF EXISTS sqlalchemy_levelup"
! mysql -e "CREATE DATABASE IF NOT EXISTS sqlalchemy_levelup"

# What is SQLAlchemy?

From [their website](http://www.sqlalchemy.org/):

> SQLAlchemy is the Python SQL toolkit and Object Relational Mapper (ORM) that gives application developers the full power and flexibility of SQL

# What is the SQLAlchemy ORM?

The ORM provides a way of modelling the objects of your domain and the relationships between them in Python.

It will persist these objects and their relationships for you, using a Relational Database Management System (RDBMS, e.g. MySQL, PostgreSQL)

# Where does the ORM fit into SQLAlchemy?

The ORM is built on top of SQLAlchemy's lower-level **Expression Language** API. This API is for working directly with database constructs and expressions in Python (e.g. it contains a `Select` class for building `select` queries)

It is possible, and sometimes preferable, to use ONLY the Expression Language if you require more granular control or if the overhead of the ORM is undesirable.

Alternatively, you can use a combination of both the ORM and the Expression Language to add more granular controlls to an ORM based application if and when they are needed

# A worked example - user management

Inspired by our Authentication service Thug Lyf

### Data model requirements
The domain objects are Users. Users will have attributes for their email address and password

# Defining the data model

As mentioned above, the ORM looks after the persistence of our domain objects.

Essentially this means we will create a Python class representing a `User`. Instances of this class will have attributes for email address and password. The ORM will store a representation of instantiated `User` objects as records in a "users" table in the database

The classes we add for our domain objects are referred to as *models* or *model classes*.

## Declarative Base Class

To have the ORM look after the persistence of our domain objects, we define our model classes as subclasses of a base class from SQLAlchemy.

This base class is provided by SQLAlchemy through the `declarative_base` class factory (its a function that returns a Python **class**)

It is common to have just one `Base` class in your application. You often see it defined in the module scope, allowing other modules to import it as a "singleton".

In [ ]:
# Create a declarative base
Base = sqlalchemy.ext.declarative.declarative_base()

## The `User` Model

In [ ]:
class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)
    
    def __repr__(self):
        return "User(email={})".format(self.email)

Note the distinction between the singular "User" classname and the "users" table name. This is because an instance of this class is a "user" but the table can contain many users

Side-note: You might have noticed that the "type" of the columns here is either `Integer` or `String`, where elsewhere you may have seen specific `VARCHAR`, `CHAR`, `BIGINT` etc. (imported from e.g. `sqlalchemy.dialects.mysql`). The difference here is that `Integer` and `String` are generic "SQL" types, which will be represented as `VARCHAR`, `INT` etc. in the database. Using these generic types will make the application more independent from the choice of RDBMS.

### Table objects

When we declare a subclass of the declarative base (i.e. a model), the ORM will create a `Table` object and a mapping between the model attributes (id, email, password) and the `Table` columns.

`Table` is the SQLAlchemy abstraction representing RDBMS tables

In [ ]:
# The table object associated with User is accessed by the __table__ class attribute
User.__table__

### Metadata

The declarative base has a `Metadata` object attached to it.

`Metadata` contains a registry of all the `Table` objects created when model classes are declared

We can access this registry under the `tables` attribute of the `metadata` object

In [ ]:
Base.metadata.tables

The registry `Metadata` maintains will be useful in a minute when we want to create actual RDBMS tables

# Connecting to the database

In SQLAlchemy the `Engine` is the ultimate source of database connectivity. 

All SQLAlchemy applications will call upon an `Engine` instance at some point in order to create a `Connection` object. This is done through the ```engine.connect()``` method.

Note that this means that SQLAlchemy does not establish an actual connection with the database until it is requested 

In [ ]:
database_url = sqlalchemy.engine.url.URL(drivername='mysql', database=DB_NAME,
                                         query={'read_default_file': '~/.my.cnf'})
print database_url

In [ ]:
engine = sqlalchemy.engine.create_engine(database_url, echo=True)  # echo=True logs SQL queries to stdout

## Creating tables

Now we have an engine to provide us with database connectivity, we can use the `Metadata` object to create database tables

In [ ]:
Base.metadata.create_all(engine)

# A Session with the ORM

Now that we have models, tables and an engine, we are ready to use the ORM to create and store `User` objects.

The ORM abstraction for managing synchronisation with the database is the **session**.

In [ ]:
Session = sqlalchemy.orm.sessionmaker(bind=engine)

First, we create a `Session` class, configured to use the `Engine` object we created when it requires a database connection. `Session` objects will not ask for connections until they need them.

Now we can create a session by calling the `Session` class:

In [ ]:
session = Session()

And add our first user object

In [ ]:
ben = User(email="ben@example.com", password="password")
print ben

session.add(ben)

The session can be thought of as a collection of objects. You can iterate over the session to access the objects within:

In [ ]:
[obj for obj in session]

And check that your object is in the session:

In [ ]:
ben in session

At this point, the user object exists in the session, but is not in the database (note that no SQL was emitted by the `echo`). To ask the ORM to flush changes to the database, we can use the `flush` method.

In [ ]:
session.flush()

`BEGIN` means a new transaction has been started on session's connection with the database.

Right now the new record is in the database's transaction buffer (and therefore not visible to anyone else). To finish the transaction and fully persist the data, we can use `commit`

In [ ]:
session.commit()

NB. We may also `commit` straightaway, without explicitly calling `flush` - the session will do that for us

## Querying

Querying for data is achieved by using the `Session` object's `query` attribute

In [ ]:
session.query(User).filter(User.email=="ben@example.com").all()

The `Session.query` method returns a `Query` object, which has methods such as `filter`, `order_by`, `limit` etc. These methods are *generative* - they return new `Query` objects allowing you to call additional methods. This is allows you to build up queries in long chains of method calls.

I won't go further into querying now, it's fairly intuitive if you are already familiar with SQL. See the [SQLAlchemy docs for the `Query` API](http://docs.sqlalchemy.org/en/latest/orm/query.html) for more.

## Updating

If we wish to modify an object which has been placed in the session, we can just directly change the desired attribute's value. The ORM will detect the change and translate it into an `UPDATE` statement at the next `flush`

Let's say we want to improve Ben's password

In [ ]:
ben.password = "drowssap"

In [ ]:
session.commit()

Any changes made to any objects in the session will be tracked by the ORM until a `flush` occurs.

Note that if the session is queried after an object has been modified, but before `flush` has been called, the ORM will issue a `flush` itself. This can bee seen in the example below, where the `UPDATE` statement happens before the `SELECT ... WHERE users.password=` statement

In [ ]:
ben.password = "new_password"

session.query(User).filter(User.password=="new_password").all()

In [ ]:
# Start fresh
session.close()  # ends the current session
Base.metadata.drop_all(bind=engine)  # drops all tables

# Extending the data model - Organisations

### Data model requirements

Our "customers" at GrowthIntel are businesses, not individuals. Because of this, our Auth service groups Users into larger units, which we call Organisations.

Organisations can have many Users. Users belong to one and only one Organisation (1 to many relationship)

### Diagram

```
      org1               org2
    /  |   \            /   \
user1 user2 user3    user4  user5
```

We can tell the ORM to handle relationships for us, with the `relationship` directive.

In order to creat an ORM relationship between `User` and `Organisation`, we need two things:

- A foreign key on `User` to specify the `Organisation` it belongs to
- An attribute we can use to refer to the related object(s) on each class

In [ ]:
Base = sqlalchemy.ext.declarative.declarative_base()

class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    # New - foreign key to organisations table
    organisation_id = sqlalchemy.Column(
        sqlalchemy.Integer,
        sqlalchemy.ForeignKey('organisations.id'))

    # New - Define the ORM relationship
    organisation = sqlalchemy.orm.relationship("Organisation",
                                               back_populates="users")
    
    
    def __repr__(self):
        return "User(email={})".format(self.email)

class Organisation(Base):
    __tablename__ = "organisations"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    
    users = sqlalchemy.orm.relationship("User",
                                        back_populates="organisation")
    
    def __repr__(self):
        return "Organisation(name={})".format(self.name)

In [ ]:
Base.metadata.create_all(bind=engine)

Now I can create Ben in an organisation Example.com, and give him some colleagues

In [ ]:
example_com = Organisation(name="example.com")

ben = User(email="ben@example.com",
           password="password1",
           organisation=example_com)

jerry = User(email="jerry@example.com",
             password="password2",
             organisation=example_com)

ellie = User(email="ellie@example.com",
             password="password3",
             organisation=example_com)

When I now add these objects to a session and commit, the ORM will handle the "details" of foreign keys for me 

In [ ]:
session = Session()

session.add_all([example_com, ben, jerry, ellie])

session.commit()

Using the ORM relationships allow us to play around with the relationships between objects in a way that would normally require a lot of faffy SQL `UPDATE`, `SELECT .. JOIN` statements

For example, we can treat the `example_com.users` attribute as an ordinary mutable Python collection of objects

In [ ]:
example_com.users

In [ ]:
example_com.users.remove(ben)

In [ ]:
rosie = User(email="rosie@example.com", password="password4")

example_com.users.append(rosie)

In [ ]:
session.flush()

In [ ]:
example_com.users

In [ ]:
rosie.organisation

In [ ]:
test_com = Organisation(name="test.com")
ben.organisation = test_com

In [ ]:
session.flush()

In [ ]:
test_com.users

In [ ]:
session.close()
Base.metadata.drop_all(bind=engine)

# Many-to-Many relationships - User Roles

### Data model requirements

User's have permissions known as roles. There are a set number of roles, and any user can have 0, 1 or many of them.

### Diagram

<img src="images/user-roles.png" />

This implies a many-to-many relationship, which is not quite as straightforward to set up with the ORM, but just as easy to use.

The complicating factor is that we can't use simple foreign key relationships, as in effect we need many foreign keys per `User`.

This is commonly solved in a relational database using a third *association table* which contains pairs of foreign keys pointing to the tables being related `user_id`-`role_id`

In [ ]:
Base = sqlalchemy.ext.declarative.declarative_base()

# NEW - association table
user_roles = sqlalchemy.Table(
    'user_roles', Base.metadata,
    sqlalchemy.Column('user_id', sqlalchemy.ForeignKey('users.id'), primary_key=True),
    sqlalchemy.Column('role_id', sqlalchemy.ForeignKey('roles.id'), primary_key=True),
)

class User(Base):
    __tablename__ = "users"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    email = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    password = sqlalchemy.Column(sqlalchemy.String(255), nullable=False)

    # NEW - relationship to Role via secondary table
    roles = sqlalchemy.orm.relationship("Role",
                                        secondary=user_roles,
                                        back_populates="users")
    
    def __repr__(self):
        return "User(email={})".format(self.email)
    
class Role(Base):
    __tablename__ = "roles"
    id = sqlalchemy.Column(sqlalchemy.Integer, primary_key=True)
    name = sqlalchemy.Column(sqlalchemy.String(255), nullable=False, unique=True)
    
    users = sqlalchemy.orm.relationship("User",
                                        secondary=user_roles,
                                        back_populates="roles")

There's a couple of things to note.

We didnt use a model class for the association table, instead directly using the `Table` object. This is because a 'user_role' is not a domain object to be created and modified in our application. Its more of a second class citizen which only the ORM needs to know about.

Second, we give the new `Table` access to the `Metadata` object, so it is tracked in its registry and all `create_all`, `drop_all` commands will include this table.

In [ ]:
Base.metadata.create_all(bind=engine)

Let's create a set of `Role` and `User` objects to play around with

In [ ]:
ben = User(email="ben@example.com",
           password="password1")

jerry = User(email="jerry@example.com",
             password="password2")

ellie = User(email="ellie@example.com",
             password="password3")

In [ ]:
admin = Role(name="admin")
test_user = Role(name="test_user")
can_export = Role(name="can_export")

In [ ]:
ben.roles.extend([admin, can_export])
jerry.roles.append(test_user)
ellie.roles.append(can_export)

In [ ]:
session.add_all([ben, jerry, ellie])

In [ ]:
session.commit()

In [ ]:
can_export.users

In [ ]:
session.close()

# Testing Applications with the ORM

There's a pattern described in the [SQLAlcheny documentation](http://docs.sqlalchemy.org/en/rel_1_1/orm/session_transaction.html#joining-a-session-into-an-external-transaction-such-as-for-test-suites) which makes use of database transactions and rollbacks to give you fully isolated database tests. We use this pattern in Thug Lyf to reset database state between each test method. This is desirable, because (a) wiping out the database and re-deploying fixture data would make tests very slow and (b) tracking the state of rows changed is somethings infeasible.

The pattern is called "Joining a session into an external transaction". It works by binding a `Session` to a `Connection` (rather than an `Engine`) which is in a "transactional state".

Whatever you do in this session will be undone when the `Connection` object's transaction is rolled back - even if you call `session.commit()`

### Demonstration

Get a `Connection` by calling `connect` on the `Engine`

In [ ]:
connection = engine.connect()

Start the transaction by calling `begin` on the `Connection` object

In [ ]:
transaction = connection.begin()

Now bind a session to the connection

In [ ]:
session = Session(bind=connection) # Overrides the bind=engine configuration we setup 
                                   # with the sessionmakeer

In [ ]:
users = session.query(User).all()

In [ ]:
users

In [ ]:
ben = users[0]
ben

In [ ]:
ben.email = "notben@example.com" 

In [ ]:
session.commit()

Note that no `COMMIT` statement was emitted. We can query for Ben, and will find that his email has apparently changed

In [ ]:
session.query(User).filter(User.email=="notben@example.com").all()

But if you check the database, nothing is apparently different. To "tear down" our changes, we only need to close the session and rollback the transaction.

In [ ]:
session.close()

In [ ]:
transaction.rollback()

We also close the connection, to start fresh with a new connection for the next test

In [ ]:
connection.close()